# 06 — Multi-Agent Orchestrator

Ein Supervisor-Agent, der freie Benutzeranfragen entgegennimmt und Spezialagenten koordiniert:
- **Compliance Classifier** → Risikostufen-Klassifizierung
- **Obligation Advisor** → Compliance-Checkliste
- **Regulation Q&A** → offene Fragen

Demonstriert das **"Agents as Tools"**-Muster — jeder Spezialist wird als aufrufbares Tool für den Orchestrator gewrappt.

Beispiel: *"Ich baue ein Gesichtserkennungssystem für die Flughafensicherheit, wir sind der Anbieter. Was muss ich wissen?"* → Orchestrator ruft Classifier auf (Hochrisiko), leitet an Obligation Advisor weiter (Anbieter-Checkliste), nutzt Q&A für weitere Fragen, und erstellt einen zusammenfassenden Bericht.

In [ ]:
import sys
sys.path.insert(0, "..")

import json
from src.tools import init_tools, TOOL_SCHEMAS, TOOL_DISPATCH
from src.agent import run_agent
from src.embeddings import get_openai_client

LANG = "de"  # change to "en" for English

client = get_openai_client()
init_tools(chroma_path="../chroma_db", lang=LANG)

## Spezialagenten als Tools definieren

In [ ]:
# System-Prompts für Spezialagenten
CLASSIFIER_PROMPT = """Du bist ein Klassifikator für die EU-KI-Verordnung. Bestimme die Risikostufe des KI-Systems.
Prüfe Art. 5 (verboten), Art. 6 + Anhang III (Hochrisiko), Art. 50 (Transparenz/begrenzt).
Nutze die Tools für den Verordnungstext. Antworte mit knappem JSON: risk_level, confidence, reasoning, 
prohibited_check, high_risk_check, transparency_check und citations. Antworte auf Deutsch."""

ADVISOR_PROMPT = """Du bist ein Pflichten-Berater für die EU-KI-Verordnung. Erstelle eine Compliance-Checkliste für die gegebene Rolle und Risikostufe.
Nutze die Tools für die tatsächlichen Pflichten. Antworte mit knappem JSON: role, risk_level, obligations 
(je mit obligation, article, description, priority), summary und citations. Antworte auf Deutsch."""

QA_PROMPT = """Du bist ein Experte für die EU-KI-Verordnung. Beantworte die Frage mithilfe der Tools.
Verwende Inline-Zitate [Art. X]. Antworte mit knappem JSON: answer, citations, confidence. Antworte auf Deutsch."""


def classify_ai_system(system_description: str) -> str:
    """Run the classifier agent on a system description."""
    response, _ = run_agent(
        client=client,
        system_prompt=CLASSIFIER_PROMPT,
        user_message=f"Klassifiziere dieses KI-System:\n\n{system_description}",
        tools=TOOL_SCHEMAS,
    )
    return response


def get_obligations(role: str, risk_level: str, system_context: str = "") -> str:
    """Run the obligation advisor agent."""
    msg = f"Rolle: {role}\nRisikostufe: {risk_level}"
    if system_context:
        msg += f"\nKontext: {system_context}"
    response, _ = run_agent(
        client=client,
        system_prompt=ADVISOR_PROMPT,
        user_message=msg,
        tools=TOOL_SCHEMAS,
    )
    return response


def ask_regulation(question: str) -> str:
    """Run the Q&A agent on a question."""
    response, _ = run_agent(
        client=client,
        system_prompt=QA_PROMPT,
        user_message=question,
        tools=TOOL_SCHEMAS,
    )
    return response

In [ ]:
# Orchestrator tool schemas — each specialist agent is a tool
ORCHESTRATOR_TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "classify_ai_system",
            "description": "Classify an AI system's risk level under the EU AI Act. Returns risk classification with citations.",
            "parameters": {
                "type": "object",
                "properties": {
                    "system_description": {"type": "string", "description": "Description of the AI system to classify"},
                },
                "required": ["system_description"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_obligations",
            "description": "Get compliance obligations for a given role and risk level. Returns a checklist with article citations.",
            "parameters": {
                "type": "object",
                "properties": {
                    "role": {"type": "string", "enum": ["provider", "deployer", "importer", "distributor"]},
                    "risk_level": {"type": "string", "enum": ["unacceptable", "high", "limited", "minimal"]},
                    "system_context": {"type": "string", "description": "Additional context about the AI system"},
                },
                "required": ["role", "risk_level"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "ask_regulation",
            "description": "Ask any question about the EU AI Act. Returns an answer with article citations.",
            "parameters": {
                "type": "object",
                "properties": {
                    "question": {"type": "string", "description": "Question about the EU AI Act"},
                },
                "required": ["question"],
            },
        },
    },
]

# Dispatch for orchestrator tools
ORCHESTRATOR_DISPATCH = {
    "classify_ai_system": lambda args: classify_ai_system(**args),
    "get_obligations": lambda args: get_obligations(**args),
    "ask_regulation": lambda args: ask_regulation(**args),
}

In [ ]:
ORCHESTRATOR_SYSTEM_PROMPT = """Du bist ein EU-KI-Verordnung Compliance-Orchestrator. Du koordinierst Spezialagenten für umfassende regulatorische Beratung.

Verfügbare Spezialagenten (als Tools nutzen):
1. classify_ai_system — Risikostufe eines KI-Systems klassifizieren
2. get_obligations — Compliance-Checkliste für Rolle + Risikostufe erstellen
3. ask_regulation — Beliebige Frage zur EU-KI-Verordnung beantworten

Workflow für eine umfassende Analyse:
1. Zuerst: Risikostufe des KI-Systems mit classify_ai_system bestimmen
2. Dann: Pflichten für die Rolle des Nutzers mit get_obligations abrufen (Ergebnis der Klassifizierung nutzen)
3. ask_regulation für zusätzliche spezifische Fragen nutzen
4. Alle Ergebnisse in einer klaren Zusammenfassung synthetisieren

Deine finale Antwort soll ein gut strukturierter Bericht sein, der alle Spezialisten-Ergebnisse kombiniert.
Antworte auf Deutsch."""

## Vollständige Orchestrierung: Gesichtserkennung am Flughafen

In [ ]:
%%time
from datetime import datetime

AGENT_LABELS = {
    "classify_ai_system": "Compliance Classifier",
    "get_obligations": "Obligation Advisor",
    "ask_regulation": "Regulation Q&A",
}

def live_tracker(event, data):
    label = AGENT_LABELS.get(data["name"], data["name"])
    ts = datetime.fromtimestamp(data["timestamp"]).strftime("%H:%M:%S")
    if event == "tool_call_start":
        args_summary = ", ".join(f"{k}={repr(v)[:60]}" for k, v in data["arguments"].items())
        print(f"[{ts}] → {label} gestartet ({args_summary})")
    elif event == "tool_call_end":
        print(f"[{ts}] ✓ {label} fertig ({data['duration']:.1f}s)")

response, messages = run_agent(
    client=client,
    system_prompt=ORCHESTRATOR_SYSTEM_PROMPT,
    user_message="""Ich baue ein Gesichtserkennungssystem für die Sicherheitskontrolle am Flughafen.
Wir sind der Anbieter dieses Systems. Das System verifiziert die Identität von Passagieren, 
indem es ihr Gesicht mit dem Passfoto am Boarding-Gate abgleicht.

Was muss ich bezüglich der EU-KI-Verordnung beachten?""",
    tools=ORCHESTRATOR_TOOLS,
    tool_dispatch=ORCHESTRATOR_DISPATCH,
    max_turns=15,
    on_tool_call=live_tracker,
)

print(f"\n{'='*80}")
print("ORCHESTRATOR-BERICHT")
print('='*80)
print(response)

## Orchestrierung: KI-gestützte medizinische Diagnose

In [ ]:
%%time

response2, messages2 = run_agent(
    client=client,
    system_prompt=ORCHESTRATOR_SYSTEM_PROMPT,
    user_message="""Wir setzen in unserem Krankenhaus ein KI-Diagnosetool ein, das medizinische Bilder 
(Röntgen, MRT) analysiert, um potenzielle Krebserkrankungen zu erkennen. Wir betreiben ein 
Drittsystem, wir entwickeln es nicht selbst.

Welche Pflichten haben wir? Gibt es spezifische Anforderungen für medizinische KI?""",
    tools=ORCHESTRATOR_TOOLS,
    tool_dispatch=ORCHESTRATOR_DISPATCH,
    max_turns=15,
    on_tool_call=live_tracker,
)

print(f"\n{'='*80}")
print("ORCHESTRATOR-BERICHT")
print('='*80)
print(response2)